In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.2.4
2.5


In [3]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [4]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [6]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [7]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x000001DA6C29FA60>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x000001DA6C29FA60>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x000001DA6C29FA60>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0

In [8]:
#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [04:07<00:00,  1.24s/it]
[MainProcess/INFO] experiments finished


In [9]:
#observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,122,159.479892,1.0,0.525221,115.372421,1.5,0.259669,181.361731,10,0.639245,...,5,8,4,2,5,6,3,4,0,dikesnet
1,26,173.068484,1.5,0.553683,68.990107,1.0,0.074385,59.815284,10,0.859599,...,5,8,4,2,5,6,3,5,0,dikesnet
2,50,165.564272,1.0,0.020646,34.162367,1.5,0.859324,155.515218,10,0.170598,...,5,8,4,2,5,6,3,6,0,dikesnet
3,34,342.470710,1.0,0.589452,286.940201,1.0,0.422351,43.964393,1.0,0.355908,...,5,8,4,2,5,6,3,7,0,dikesnet
4,86,154.851051,10,0.694219,191.458671,1.0,0.746860,106.281865,1.0,0.692390,...,5,8,4,2,5,6,3,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,45,52.187596,1.0,0.312793,254.692985,10,0.954835,342.170192,1.5,0.641922,...,9,5,7,6,7,8,1,49,3,dikesnet
196,77,191.182254,10,0.019366,313.818802,1.0,0.123258,175.034277,1.0,0.892232,...,9,5,7,6,7,8,1,50,3,dikesnet
197,104,234.454320,1.0,0.679455,295.006489,1.0,0.998729,55.047523,1.5,0.140889,...,9,5,7,6,7,8,1,51,3,dikesnet
198,28,215.913093,1.5,0.787277,49.090250,10,0.108844,288.899163,10,0.807560,...,9,5,7,6,7,8,1,52,3,dikesnet


In [23]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{'0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 2', **{'0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.1_DikeIncrease 0':5}),
            Policy('policy 3', **{'0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.1_DikeIncrease 0':10}),
            Policy('policy 4', **{'0_RfR 0':1,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 5', **{'0_RfR 0':0,
                                  '0_RfR 1':1,
                                  '0_RfR 2':0,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 6', **{'0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':1,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 7', **{'1_RfR 0':1,
                                  '1_RfR 1':0,
                                  '1_RfR 2':0,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 8', **{'1_RfR 0':0,
                                  '1_RfR 1':1,
                                  '1_RfR 2':0,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 9', **{'1_RfR 0':0,
                                  '1_RfR 1':0,
                                  '1_RfR 2':1,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 10', **{'2_RfR 0':1,
                                  '2_RfR 1':0,
                                  '2_RfR 2':0,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 11', **{'2_RfR 0':1,
                                  '2_RfR 1':0,
                                  '2_RfR 2':0,
                                  'A.1_DikeIncrease 0':0}),
            Policy('policy 12', **{'2_RfR 0':1,
                                  '2_RfR 1':0,
                                  '2_RfR 2':0,
                                  'A.1_DikeIncrease 0':0}),
                                ]


In [24]:
#pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 12 policies * 1 model(s) = 1200 experiments
100%|██████████████████████████████████████| 1200/1200 [14:21<00:00,  1.39it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [27]:
import pandas as pd
a, b = results
df = pd.DataFrame(b)
df['policy']=pd.DataFrame(a)['policy']
df

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,policy
0,1.526726e+08,0.025022,1.229878e+08,0.003240,5.894588e+07,0.000000,2.349976e+07,0.000871,7.454585e+07,0.000000,0.0,1082.473734,policy 1
1,6.090746e+07,0.000000,8.531604e+08,0.241852,5.894588e+07,0.000000,1.858668e+07,0.000000,7.454585e+07,0.000000,0.0,21681.104930,policy 1
2,6.090746e+07,0.000000,1.145610e+08,0.000529,8.085810e+07,0.011498,7.860204e+07,0.008197,7.454585e+07,0.000000,0.0,2168.508371,policy 1
3,7.236820e+08,0.154260,1.858661e+08,0.026593,2.155391e+08,0.086036,1.451749e+08,0.019498,7.454585e+07,0.000000,0.0,14123.004437,policy 1
4,3.861073e+09,0.724307,1.126626e+08,0.000000,5.894588e+07,0.000000,1.858668e+07,0.000000,7.454585e+07,0.000000,0.0,16576.281398,policy 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,5.714000e+07,0.000000,9.640617e+08,0.306146,5.574000e+07,0.000000,1.623000e+07,0.000000,8.435463e+07,0.005897,30700000.0,29487.650558,policy 12
1196,9.308237e+07,0.006935,1.022563e+08,0.000000,1.352738e+08,0.038452,2.082691e+07,0.000665,2.522035e+08,0.045227,30700000.0,5849.751953,policy 12
1197,1.033269e+08,0.011223,4.868215e+08,0.115280,5.574000e+07,0.000000,1.897407e+07,0.000423,6.813000e+07,0.000000,30700000.0,10294.503382,policy 12
1198,2.642641e+09,0.722597,1.022563e+08,0.000000,6.015287e+07,0.003040,1.623000e+07,0.000000,6.813000e+07,0.000000,30700000.0,16686.662169,policy 12


In [26]:
def q05(x):
    return x.quantile(0.05)
def q95(x):
    return x.quantile(0.95)

In [28]:
group_df = df.groupby(by="policy")
agg_par = ['mean', 'min', 'max', q05, q95]
agg_dict = {
    'A.3 Total Costs': agg_par,
    'A.3_Expected Number of Deaths': agg_par,
    'RfR Total Costs':agg_par,
    'Expected Evacuation Costs':agg_par
}
TCA_df = group_df.agg(agg_dict)
TCA_df

A.3 Total Costs                                            \
                     mean           min           max           q05   
policy                                                                
policy 1     4.728708e+08  5.574000e+07  2.810428e+09  5.574000e+07   
policy 10    3.141603e+08  5.574000e+07  2.660424e+09  5.574000e+07   
policy 11    3.141603e+08  5.574000e+07  2.660424e+09  5.574000e+07   
policy 12    3.141603e+08  5.574000e+07  2.660424e+09  5.574000e+07   
policy 2     6.127342e+08  5.574000e+07  2.810397e+09  5.574000e+07   
policy 3     6.166441e+08  5.574000e+07  2.810397e+09  5.574000e+07   
policy 4     5.617347e+08  5.574000e+07  2.810397e+09  5.574000e+07   
policy 5     5.617347e+08  5.574000e+07  2.810397e+09  5.574000e+07   
policy 6     5.617347e+08  5.574000e+07  2.810397e+09  5.574000e+07   
policy 7     5.593952e+08  5.574000e+07  2.810397e+09  5.574000e+07   
policy 8     5.593952e+08  5.574000e+07  2.810397e+09  5.574000e+07   
policy 9     5.593952e+08  5.574000e+07  2.810397e+09  5.574000e+07   

                        A.3_Expected Number of Deaths                      \
                    q95                          mean  min       max  q05   
policy                                                                      
policy 1   2.294751e+09                      0.256116  0.0  1.356421  0.0   
policy 10  2.023827e+09                      0.152561  0.0  1.255861  0.0   
policy 11  2.023827e+09                      0.152561  0.0  1.255861  0.0   
policy 12  2.023827e+09                      0.152561  0.0  1.255861  0.0   
policy 2   2.292308e+09                      0.349618  0.0  1.356421  0.0   
policy 3   2.292308e+09                      0.352171  0.0  1.356421  0.0   
policy 4   2.292308e+09                      0.314975  0.0  1.356421  0.0   
policy 5   2.292308e+09                      0.314975  0.0  1.356421  0.0   
policy 6   2.292308e+09                      0.314975  0.0  1.356421  0.0   
policy 7   2.292308e+09                      0.313371  0.0  1.356421  0.0   
policy 8   2.292308e+09                      0.313371  0.0  1.356421  0.0   
policy 9   2.292308e+09                      0.313371  0.0  1.356421  0.0   

                    RfR Total Costs                                         \
                q95            mean          min          max          q05   
policy                                                                       
policy 1   1.328432             0.0          0.0          0.0          0.0   
policy 10  1.234996      30700000.0   30700000.0   30700000.0   30700000.0   
policy 11  1.234996      30700000.0   30700000.0   30700000.0   30700000.0   
policy 12  1.234996      30700000.0   30700000.0   30700000.0   30700000.0   
policy 2   1.333095             0.0          0.0          0.0          0.0   
policy 3   1.333095             0.0          0.0          0.0          0.0   
policy 4   1.331717      84600000.0   84600000.0   84600000.0   84600000.0   
policy 5   1.331717      84600000.0   84600000.0   84600000.0   84600000.0   
policy 6   1.331717      84600000.0   84600000.0   84600000.0   84600000.0   
policy 7   1.328432     217800000.0  217800000.0  217800000.0  217800000.0   
policy 8   1.328432     217800000.0  217800000.0  217800000.0  217800000.0   
policy 9   1.328432     217800000.0  217800000.0  217800000.0  217800000.0   

                       Expected Evacuation Costs                             \
                   q95                      mean          min           max   
policy                                                                        
policy 1           0.0              23622.141915  1082.473734  70796.935351   
policy 10   30700000.0              19742.703915  1423.658459  68854.791813   
policy 11   30700000.0              19742.703915  1423.658459  68854.791813   
policy 12   30700000.0              19742.703915  1423.658459  68854.791813   
policy 2           0.0              24383.228711  1628.872417  9

In [30]:
TCA_df.to_csv("results/TCA3_results.csv")